<a href="https://www.kaggle.com/code/kentvejrupmadsen/classification-of-letters?scriptVersionId=133575268" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install ipykernel
!python -m ipykernel install --user --name=notebook_environment

%pip install matplotlib
%pip install keras
%pip install tensorflow
%pip install kaggle
%pip install numpy
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Installed kernelspec notebook_environment in /root/.local/share/jupyter/kernels/notebook_environment
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from random \
    import SystemRandom

import tensorflow

from tensorflow.data \
    import AUTOTUNE

from kaggle_secrets \
    import UserSecretsClient

from keras.losses \
    import SparseCategoricalCrossentropy

from keras.utils \
    import image_dataset_from_directory

from keras \
    import \
    Model, \
    Sequential

from keras \
    import layers

import wandb

from wandb.integration.keras \
    import \
    WandbCallback, \
    WandbEvalCallback, \
    WandbMetricsLogger, \
    WandbModelCheckpoint

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
secrets = UserSecretsClient()

In [4]:
random_generator = SystemRandom()

def get_max_random_value() -> int:
    return 16777215

def get_min_random_value() -> int:
    return 0

def generate_random_seed() -> int:
    global random_generator
    
    return random_generator.randint(
        get_min_random_value(), 
        get_max_random_value()
    )

In [5]:
# Constants
def label_training() -> str:
    return 'training'

def label_validation() -> str:
    return 'validation'

def empty_string() -> str:
    return ''

def zero() -> int:
    return 0

def get_log_runs() -> bool:
    global log_runs
    return log_runs

def get_gpus_to_use() -> list:
    return [ '/gpu:0', '/gpu:1' ]

# Dynamic
def set_size_of_training_set_categories(
    value: int
) -> None:
    global configuration
    configuration['dataset']['training']['size'] = value

def set_labels_of_training_set(
    value: list
) -> None:
    global configuration
    configuration['dataset']['training']['labels'] = value
    set_size_of_training_set_categories(
        len(
            value
        )
    )

def get_size_of_training_set_categories() -> int:
    global configuration
    return configuration['dataset']['training']['size']



def set_size_of_validation_set_categories(
    value: int
) -> None:
    global configuration
    configuration['dataset']['validation']['size'] = value


def set_labels_of_validation_set(
    value: list
) -> None:
    global configuration
    configuration['dataset']['validation']['labels'] = value
    
    set_size_of_validation_set_categories(
        len(
            value
        )
    )


def get_size_of_validation_set_categories() -> int:
    global configuration
    return configuration['dataset']['validation']['size']

In [6]:
__wandb_key: str = secrets.get_secret("__wandb__")

    
def get_wandb_secret() -> str:
    global __wandb_key
    
    if __wandb_key is None:
        return empty_string()
    
    return __wandb_key


In [7]:
configuration = {
    'batch_size': 15,
    'epochs': 2,
    'view': {
        'height': 512,
        'width': 512,
        'channels': 3
    },
    'dataset':
    {
        'split_validation_at': 0.45,
        'training':
        {
            'labels': [],
            'size': zero()
        },
        'validation':
        {
            'labels': [],
            'size': zero()
        }
    },
    'seed': generate_random_seed()
}

In [8]:
#
use_cache: bool = False
log_runs: bool = False

epochs = configuration['epochs']
cache_buffer_size: int = AUTOTUNE

vision_height: int = configuration['view']['height']
vision_width: int = configuration['view']['width']

vision_number_of_color_channels: int = configuration['view']['channels']

vision_size = (vision_height, vision_width) 
vision_size_w_channels = (vision_height, vision_width, vision_number_of_color_channels)

split_dataset_at: float = configuration['dataset']['split_validation_at']

dataset_seed: int = configuration['seed']
dataset_batch_size: int = configuration['batch_size']

path_to_model: str = '/kaggle/working/model'
path_to_dataset: str = '/kaggle/input/letter-images-dataset/dataset'

training_dataset = None
validation_dataset = None

pointer_to_training_dataset = None
pointer_to_validation_dataset = None

model = None
    
# Callbacks
wandb_callback = None

fit_callbacks = []

# result
results: list = []

strategy = tensorflow.distribute.MirroredStrategy(
    devices=get_gpus_to_use()
)

In [9]:
def start_process() -> None:
    if get_log_runs():
        wandb.login(
            key=get_wandb_secret()
        )
        
def initialise_process() -> None:
    if get_log_runs():
        wandb.init(
            project = 'letter-identification',
            entity = 'designermadsen',    
            config = configuration,
            sync_tensorboard = True,
            save_code = True,
            monitor_gym = True
        )

def end_process() -> None:
    if get_log_runs():
        wandb.finish()


In [10]:
start_process()

training_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset= label_training(),
    seed = dataset_seed,
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle = True,
    crop_to_aspect_ratio = True
)

set_labels_of_training_set(
    training_dataset.class_names
)

validation_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset = label_validation(),
    seed = dataset_seed,
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle = True,
    crop_to_aspect_ratio = True
)

set_labels_of_validation_set(
    validation_dataset.class_names
)

Found 11339 files belonging to 67 classes.
Using 6237 files for training.
Found 11339 files belonging to 67 classes.
Using 5102 files for validation.


In [11]:
initialise_process()

In [12]:
def make_model():
    global vision_size_w_channels
    
    division = (2, 2)
    
    return \
        Sequential(
            [
                layers.Rescaling(
                    1./255, 
                    input_shape=vision_size_w_channels, 
                    trainable=True
                ),
                layers.Conv2D(
                    256, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.MaxPooling2D(
                    pool_size=division
                ),
                layers.Conv2D(
                    128, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.MaxPooling2D(
                    pool_size=division
                ),
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    64, 
                    3,
                    padding='same', 
                    activation='relu'
                ),
                layers.MaxPooling2D(
                    pool_size=division
                ),
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    64,
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.MaxPooling2D(
                    pool_size=division
                ),
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.MaxPooling2D(
                    pool_size=division
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.MaxPooling2D(
                    pool_size=division
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                layers.MaxPooling2D(
                    pool_size=division
                ),
                layers.Flatten(),
                layers.Dense(
                    (get_size_of_training_set_categories() * 8), 
                    activation='relu',
                ),
                layers.Dense(
                    get_size_of_training_set_categories()
                )
            ]
        )

In [13]:
if not (strategy is None):
    with strategy.scope():
        model = make_model()
else:
    model = make_model()

In [14]:
model.compile(
    optimizer='adam', 
    loss=SparseCategoricalCrossentropy(
        from_logits=True
    ),
    metrics=['accuracy'], 
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 512, 512, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 256)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 128)     295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 128)    0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      7

In [15]:
if use_cache:
    pointer_to_training_dataset = training_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
    
    pointer_to_validation_dataset = validation_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
else:
    pointer_to_training_dataset = training_dataset
    pointer_to_validation_dataset = validation_dataset

In [16]:
if get_log_runs():
    wandb_callback = WandbCallback(
        log_weights=True, 
        log_gradients=True, 
        training_data=pointer_to_training_dataset, 
        validation_data=pointer_to_validation_dataset,

        log_evaluation=True, 

        monitor='val_accuracy',
        mode='max'
    )

In [17]:
if not (wandb_callback is None):
    fit_callbacks.append(wandb_callback)

In [18]:
history = None

In [19]:
if(len(fit_callbacks) > 0):
    history = model.fit(
        pointer_to_training_dataset,
        validation_data = pointer_to_validation_dataset,
        epochs=epochs,
        callbacks=fit_callbacks
    )
else:
    history = model.fit(
        pointer_to_training_dataset,
        validation_data = pointer_to_validation_dataset,
        epochs=epochs
    )

Epoch 1/2
416/416 [==============================] - 264s 546ms/step - loss: 3.9066 - accuracy: 0.0544 - val_loss: 3.8607 - val_accuracy: 0.0643
Epoch 2/2
416/416 [==============================] - 211s 506ms/step - loss: 3.8613 - accuracy: 0.0574 - val_loss: 3.8592 - val_accuracy: 0.0643


In [20]:
result = {
    'validation': {
        'accuracy': history.history['val_accuracy'],
        'loss': history.history['val_loss']
    },
    'result': {
        'accuracy': history.history['accuracy'],
        'loss': history.history['loss']
    }
}

results.append(result)

In [21]:
if (get_log_runs()):
    wandb.log( { 'training': results } )

model.save(path_to_model)

In [22]:
end_process()